In [1]:
import pandas as pd
import numpy as np

second attempt of cleaning the education and this time in a better way

In [2]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_regions_dirty.csv')
df = pd.read_csv(url)
df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.50000,1.86000,13.76,3.52,11.0,46.06,Asia
1,Afghanistan,1981,NaN,NaN,NaN,NaN,NaN,NaN,Asia
2,Afghanistan,1982,NaN,NaN,NaN,NaN,NaN,NaN,Asia
3,Afghanistan,1983,NaN,NaN,NaN,NaN,NaN,NaN,Asia
4,Afghanistan,1984,NaN,NaN,NaN,NaN,NaN,NaN,Asia
...,...,...,...,...,...,...,...,...,...
9529,Zimbabwe,2017,9.68927,8.00596,NaN,NaN,NaN,NaN,Africa
9530,Zimbabwe,2018,NaN,NaN,NaN,NaN,NaN,NaN,Africa
9531,Zimbabwe,2019,NaN,NaN,NaN,NaN,NaN,NaN,Africa
9532,Zimbabwe,2020,NaN,NaN,NaN,NaN,NaN,NaN,Africa


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9534 entries, 0 to 9533
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country Name             9534 non-null   object 
 1   Year                     9534 non-null   int64  
 2   Tertiary Enrolment (F)   4475 non-null   float64
 3   Tertiary Enrolment (M)   4475 non-null   float64
 4   Secondary Enrolment (M)  2762 non-null   float64
 5   Secondary Enrolment (F)  2762 non-null   float64
 6   Primary Enrolment (F)    3079 non-null   float64
 7   Primary Enrolment (M)    3079 non-null   float64
 8   Region                   9534 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 670.5+ KB


In [4]:
# Define the numeric columns of interest
numeric_columns = [
    'Tertiary Enrolment (F)', 'Tertiary Enrolment (M)',
    'Secondary Enrolment (M)', 'Secondary Enrolment (F)',
    'Primary Enrolment (F)', 'Primary Enrolment (M)'
]

# Step 1: Normal Interpolation (without forward or backward fill)
def interpolate_data(group):
    group[numeric_columns] = group[numeric_columns].interpolate(method='linear', limit_area='inside')
    return group

# Apply interpolation while excluding the group keys
df_interpolated = df.groupby('Country Name', group_keys=False).apply(lambda x: interpolate_data(x))

# Display the first few rows to inspect the changes
df_interpolated.head()

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65548/756159708.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_interpolated = df.groupby('Country Name', group_keys=False).apply(lambda x: interpolate_data(x))


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,Asia


In [5]:
# Step 2: Conditional Filling for Columns with Less Than 40% Null Values
threshold = 0.40
columns_to_fill = [col for col in numeric_columns if df_interpolated[col].isnull().mean() < threshold]

def fill_with_trend(group):
    for col in columns_to_fill:
        # Identify the columns that should be capped at 96%
        if 'Primary Enrolment' in col or 'Secondary Enrolment' in col:
            group[col] = group[col].apply(lambda x: min(x, 96) if pd.notnull(x) else x)
            group[col] = np.where(group[col] < 0, 0, group[col])
        else:
            group[col] = group[col].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
        
        # Forward and backward fill without replacing existing values
        group[col] = group[col].ffill().bfill()
        
        # Forward fill only if there are at most 6 consecutive missing years
        mask = group[col].isnull()
        if mask.any():
            consecutive_nulls = mask.astype(int).groupby(mask.ne(mask.shift()).cumsum()).cumsum()
            group[col] = group[col].where(~mask | (consecutive_nulls <= 6), group[col].ffill())
            
        # Check for negative trends and handle accordingly
        changes = group[col].diff().fillna(0)
        if any(changes < 0):
            group[col] = group[col].ffill()
    return group

df_filled_trend = df_interpolated.groupby('Country Name', group_keys=False).apply(lambda x: fill_with_trend(x))

# Display the first few rows to inspect the changes
df_filled_trend.head()


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65548/531651657.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled_trend = df_interpolated.groupby('Country Name', group_keys=False).apply(lambda x: fill_with_trend(x))


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,Asia


In [6]:
numeric_columns

['Tertiary Enrolment (F)',
 'Tertiary Enrolment (M)',
 'Secondary Enrolment (M)',
 'Secondary Enrolment (F)',
 'Primary Enrolment (F)',
 'Primary Enrolment (M)']

In [7]:
# Step 3: Forward Fill for Columns with Missing Values Post-2017
def forward_fill_post_2017(group):
    for col in numeric_columns:
        # Check if there are any missing values post-2017
        mask = group['Year'] > 2017
        if group.loc[mask, col].isnull().any():
            # Forward fill missing values post-2017
            group.loc[mask, col] = group.loc[mask, col].ffill()
    return group

df_forward_filled = df_filled_trend.groupby('Country Name', group_keys=False).apply(forward_fill_post_2017)

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65548/3969186216.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_forward_filled = df_filled_trend.groupby('Country Name', group_keys=False).apply(forward_fill_post_2017)


In [8]:
# Function to map country to region using pycountry and existing mapping
def get_region(country_name):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.region
    except LookupError:
        return country_region_mapping.get(country_name, "Unknown")

# If the region column is missing or needs to be verified
if 'Region' not in df_forward_filled.columns or df_forward_filled['Region'].isnull().any():
    df_forward_filled['Region'] = df_forward_filled['Country Name'].apply(get_region)

# Check the unique regions to ensure correctness
print(df_forward_filled['Region'].unique())

# Define numeric columns to fill missing values
numeric_columns = df_forward_filled.select_dtypes(include=[np.number]).columns

# Function to fill missing values with the mean of the region for each year for numeric columns only
def fill_missing_values(group):
    for col in numeric_columns:
        group[col] = group[col].fillna(group[col].mean())
    return group

# Group by 'Region' and 'Year' and fill missing values with the regional mean for that year
df_forward_filled = df_forward_filled.groupby(['Region', 'Year']).apply(fill_missing_values).reset_index(drop=True)

# Group by 'Country Name' and 'Year' and interpolate missing values linearly for numeric columns only
def interpolate_values(group):
    group[numeric_columns] = group[numeric_columns].interpolate(method='linear')
    return group

df_final = df_forward_filled.groupby('Country Name', group_keys=True).apply(interpolate_values).reset_index(drop=True)


df_final.head()


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65548/2943757477.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_forward_filled = df_forward_filled.groupby(['Region', 'Year']).apply(fill_missing_values).reset_index(drop=True)


['Asia' 'Africa' 'Europe' 'Oceania' 'North America' 'South America'
 'Various']


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65548/2943757477.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_forward_filled.groupby('Country Name', group_keys=True).apply(interpolate_values).reset_index(drop=True)


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,Asia


In [9]:
df_final.isna().sum()

Country Name               0
Year                       0
Tertiary Enrolment (F)     0
Tertiary Enrolment (M)     0
Secondary Enrolment (M)    0
Secondary Enrolment (F)    0
Primary Enrolment (F)      0
Primary Enrolment (M)      0
Region                     0
dtype: int64

In [10]:
#save the data cleanly in cleaned file
df_filled_trend.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_regions_dirty2342341234.csv',index=False,encoding='latin-1')

In [11]:
filtered_df_china = df_final[df_final['Country Name'] == 'Spain']
filtered_df_china

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
8064,Spain,1980,21.100000,27.220000,73.040000,75.000000,100.000000,99.990000,Europe
8065,Spain,1981,22.010674,27.103812,73.717330,75.716268,99.832404,99.810191,Europe
8066,Spain,1982,22.921348,26.987624,74.394660,76.432537,99.664807,99.630382,Europe
8067,Spain,1983,23.832022,26.871436,75.071990,77.148805,99.497211,99.450574,Europe
8068,Spain,1984,24.742696,26.755248,75.749320,77.865074,99.329615,99.270765,Europe
8069,Spain,1985,25.653370,26.639060,76.426650,78.581342,99.162019,99.090956,Europe
8070,Spain,1986,28.657270,28.473190,77.103980,79.297611,98.994422,98.911147,Europe
8071,Spain,1987,30.226980,29.542140,77.781310,80.013879,98.826826,98.731339,Europe
8072,Spain,1988,32.601610,30.947360,78.458640,80.730147,98.659230,98.551530,Europe
8073,Spain,1989,34.452920,33.004150,79.135970,81.446416,99.351060,99.115590,Europe


In [12]:
filtered_df_china = df_final[df_final['Country Name'] == 'Spain']
filtered_df_china

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
8064,Spain,1980,21.100000,27.220000,73.040000,75.000000,100.000000,99.990000,Europe
8065,Spain,1981,22.010674,27.103812,73.717330,75.716268,99.832404,99.810191,Europe
8066,Spain,1982,22.921348,26.987624,74.394660,76.432537,99.664807,99.630382,Europe
8067,Spain,1983,23.832022,26.871436,75.071990,77.148805,99.497211,99.450574,Europe
8068,Spain,1984,24.742696,26.755248,75.749320,77.865074,99.329615,99.270765,Europe
8069,Spain,1985,25.653370,26.639060,76.426650,78.581342,99.162019,99.090956,Europe
8070,Spain,1986,28.657270,28.473190,77.103980,79.297611,98.994422,98.911147,Europe
8071,Spain,1987,30.226980,29.542140,77.781310,80.013879,98.826826,98.731339,Europe
8072,Spain,1988,32.601610,30.947360,78.458640,80.730147,98.659230,98.551530,Europe
8073,Spain,1989,34.452920,33.004150,79.135970,81.446416,99.351060,99.115590,Europe


In [13]:
#save the data cleanly in cleaned file
df_final.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_good_clean_regions.csv',index=False,encoding='latin-1')

In [14]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/gender-gap-education-levels_good_clean_regions.csv')
df_df = pd.read_csv(url)
df_df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,Asia
...,...,...,...,...,...,...,...,...,...
9529,Zimbabwe,2017,9.689270,8.005960,44.785263,44.945318,79.964715,80.780638,Africa
9530,Zimbabwe,2018,9.689270,8.005960,48.050756,48.819893,81.027363,81.907439,Africa
9531,Zimbabwe,2019,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,Africa
9532,Zimbabwe,2020,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,Africa


In [15]:
df_clean_no_regions = df_df.drop(columns=['Region'])

In [16]:
#save the data cleanly in cleaned file
df_clean_no_regions.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/gender-gap-education-levels_cleanded.csv',index=False,encoding='latin-1')